## Cafeteria 
Початкові параметри були виміряні за допомогою наступних бібліотек:
1. Вимірювання часу (час проходження assert) виконання коду - library datetime
2. Вимірювання пам'яті - library psutil

Стартовий код, написаний студентом(без доктестів та без дотримання pep8 для тесту ШІ):

In [8]:
import psutil

def memory_usage():
    process = psutil.Process()
    mem = process.memory_info().rss / (1024 * 1024)
    return mem

class Track:

    _Track__beans = 5000
    _Track__milk = 20000
    safety = True

    MENU = {
        "espresso": 40,
        "latte": 70,
        "flat white": 70,
        "dopio": 50,
        "cappuccino": 60,
        "lungo": 50,
        "cortado": 55,
        "mocca": 60}

    def __init__(self, data):
        self.data = data
        self.orders = []
        self.revenue = 0
        self.milks = 0
        self.bean = 0

        self.milk = Track._Track__milk

    def place_order(self, order):
        if Track.safety == True:
            if isinstance(order, Coffee):
                if order.name in Track.MENU and order.name in Coffee.recipe:
                    if ('foamed_milk' in Coffee.recipe[order.name]
                            or 'steamed_milk' in Coffee.recipe[order.name]):
                        if Coffee.recipe[order.name]['foamed_milk'] > self.milk:
                            return "Unfortunately, we don't have enough ingredients."

                    order.is_paid = True
                    self.bean += order.espresso/5
                    self.revenue += order.price
                    self.milks += order.milk
                    self.orders.append(order)

                    self.beans = Track._Track__beans - self.bean
                    self.milk = Track._Track__milk - self.milks
                    return 'Done!'
                if order.name not in Track.MENU:
                    return "Unfortunately, we don't have such kind of coffee in the menu."
            else:
                return "We can't create anything that is not a Coffee instance."
        else:
            return "Unfortunately, now it is not safe to make coffee."

    @classmethod
    def set_limit_milk(cls, amount):
        cls._Track__milk = amount

    @classmethod
    def change_air_state(cls):
        if cls.safety == True:
            cls.safety = False
        else:
            cls.safety = True

    def total_revenue(self):
        return self.revenue

    def total_milk(self):
        return self.milks

    def total_beans(self):
        return self.bean

    def milk_spoil(self, amount):
        self.milk -= amount

class Coffee:
    recipe = {}
    _Coffee__recipe = {}

    def __init__(self, name, count = 1):
        self.name = name
        self.count = count
        if self.name in Coffee.recipe and self.name in Track.MENU:
            self.espresso = Coffee.recipe[self.name]['espresso'] * self.count

            if 'steamed_milk' in Coffee.recipe[self.name]:
                self.steamed_milk = Coffee.recipe[self.name]['steamed_milk'] * self.count
            else:
                self.steamed_milk = 0

            if 'foamed_milk' in Coffee.recipe[self.name]:
                self.foamed_milk = Coffee.recipe[self.name]['foamed_milk'] * self.count
            else:
                self.foamed_milk = 0

            self.milk = self.steamed_milk + self.foamed_milk


            if self.steamed_milk != 0 and self.foamed_milk != 0:
                Coffee._Coffee__recipe[self.name] = {'espresso':
                                                         Coffee.recipe[self.name]['espresso'],
                                                     'steamed_milk':
                                                         Coffee.recipe[self.name]['steamed_milk'],
                                                     'foamed_milk':
                                                         Coffee.recipe[self.name]['foamed_milk']}
            elif self.foamed_milk != 0:
                Coffee._Coffee__recipe[self.name] = {'espresso':
                                                         Coffee.recipe[self.name]['espresso'],
                                                     'foamed_milk':
                                                         Coffee.recipe[self.name]['foamed_milk']}
            elif self.steamed_milk != 0:
                Coffee._Coffee__recipe[self.name] = {'espresso':
                                                         Coffee.recipe[self.name]['espresso'],
                                                     'steamed_milk':
                                                         Coffee.recipe[self.name]['steamed_milk']}
            else:
                Coffee._Coffee__recipe[self.name] = {'espresso':
                                                         Coffee.recipe[self.name]['espresso']}

            self.price = Track.MENU[self.name] * self.count
        if Coffee.recipe != {} and self.name in Coffee.recipe:
            self.is_paid = False

    def __str__(self):
        if Coffee.recipe == {}:
            return 'Order cannot be created. Recipe has not been set.'
        if self.name not in Coffee.recipe:
            return "Order cannot be created. We don't have recipe for it."

        if self.is_paid == False:
            return f'Order "{self.count} {self.name}" is created.'
        else:
            return f'Preparing {self.count} {self.name}...'

    def __repr__(self):
        return f'{self.count} {self.name}'

    @classmethod
    def set_recipe(cls, recipe):
        cls.recipe = recipe

    def __eq__(self, other):
        if isinstance(other, CustomCoffee) and other.flavor == True:
            return False
        elif self.name == other.name and self.count == other.count:
            return True
        return False


class FlavorMixin:
    pass

class CustomCoffee(Coffee, FlavorMixin):
    def __init__(self, name, count = 1):
        super().__init__(name, count)
        self.flavor = False

    def add_flavor(self, sugar, cinammon, syrup):
        if self.is_paid == True:
            self.sugar = sugar * self.count
            self.cinammon = cinammon
            self.syrup = syrup
            self.flavor = True
            return 'Done!'
        return 'Please, pay for it.'

    def __str__(self):
        if self.flavor == False:
            if self.is_paid == False:
                return f'Order "{self.count} custom {self.name}" is created.'
            return super().__str__()
        else:
            if self.cinammon == True:
                return (f'Your best {self.name} is ready! It has: {self.sugar} stickers of sugar, '
                        f'cinammon, {self.syrup} syrup.')
            else:
                return (f'Your best {self.name} is ready! It has: {self.sugar} stickers of sugar, '
                        f'{self.syrup} syrup.')

    def __repr__(self):
        return f'{self.count} custom {self.name}'

    def __eq__(self, other):
        if isinstance(other, Coffee) and self.flavor == True:
            return False
        if self.name == other.name and self.count == other.count:
            return True
        return False

RECIPE = {
    "espresso": {
        'espresso': 30},
    "latte": {
        'espresso': 60,
        'steamed_milk': 120,
        'foamed_milk': 15},
    "macchiato": {
        'espresso': 60,
        'foamed_milk': 15},
    "flat white": {
        'espresso': 60,
        'steamed_milk': 120},
    "dopio": {
        'espresso': 60},
    "cappuccino": {
        'espresso': 60,
        'steamed_milk': 60,
        'foamed_milk': 60},
    "lungo": {
        'espresso': 90},
    "cortado": {
        'espresso': 60,
        'steamed_milk': 60}
}

Параметри коду без втручення ШІ: а) час виконання = 0.001048088 с. б) кількість пам'яті = 42.8046875 МВ.

##### Промпт 1
-> Оптимізуй цей код ... 

СhatGPT: -> Для оптимізації цього коду ми можемо здійснити кілька кроків.

In [9]:
class Track:
    __beans = 5000
    __milk = 20000
    safety = True

    MENU = {
        "espresso": 40,
        "latte": 70,
        "flat white": 70,
        "dopio": 50,
        "cappuccino": 60,
        "lungo": 50,
        "cortado": 55,
        "mocca": 60}

    def __init__(self, data):
        self.data = data
        self.orders = []
        self.revenue = 0
        self.milks = 0
        self.beans = 0
        self.milk = Track.__milk

    def place_order(self, order):
        if Track.safety:
            if isinstance(order, Coffee) and order.name in Track.MENU and order.name in Coffee.recipe:
                recipe = Coffee.recipe[order.name]
                if 'foamed_milk' in recipe or 'steamed_milk' in recipe:
                    required_milk = max(recipe.get('foamed_milk', 0), recipe.get('steamed_milk', 0))
                    if required_milk > self.milk:
                        return "Unfortunately, we don't have enough ingredients."
                order.is_paid = True
                self.beans += order.espresso / 5
                self.revenue += order.price
                self.milks += order.milk
                self.orders.append(order)
                self.beans = max(0, Track.__beans - self.beans)
                self.milk = max(0, Track.__milk - self.milks)
                return 'Done!'
            return "Unfortunately, we don't have such kind of coffee in the menu." if order.name not in Track.MENU else "We can't create anything that is not a Coffee instance."
        return "Unfortunately, now it is not safe to make coffee."

    @classmethod
    def set_limit_milk(cls, amount):
        cls.__milk = amount

    @classmethod
    def change_air_state(cls):
        cls.safety = not cls.safety

    def total_revenue(self):
        return self.revenue

    def total_milk(self):
        return self.milks

    def total_beans(self):
        return self.beans

    def milk_spoil(self, amount):
        self.milk -= amount


class Coffee:
    recipe = {}

    def __init__(self, name, count=1):
        self.name = name
        self.count = count
        self.is_paid = False
        if self.name in Coffee.recipe and self.name in Track.MENU:
            recipe = Coffee.recipe[self.name]
            self.espresso = recipe['espresso'] * self.count
            self.steamed_milk = recipe.get('steamed_milk', 0) * self.count
            self.foamed_milk = recipe.get('foamed_milk', 0) * self.count
            self.milk = self.steamed_milk + self.foamed_milk
            self.price = Track.MENU[self.name] * self.count

    def __str__(self):
        if not Coffee.recipe:
            return 'Order cannot be created. Recipe has not been set.'
        return f'Order "{self.count} {self.name}" is created.' if self.is_paid else f'Preparing {self.count} {self.name}...'

    def __repr__(self):
        return f'{self.count} {self.name}'

    @classmethod
    def set_recipe(cls, recipe):
        cls.recipe = recipe


class CustomCoffee(Coffee):
    def __init__(self, name, count=1):
        super().__init__(name, count)
        self.flavor = False

    def add_flavor(self, sugar, cinammon, syrup):
        if not self.is_paid:
            return 'Please, pay for it.'
        self.sugar = sugar * self.count
        self.cinammon = cinammon
        self.syrup = syrup
        self.flavor = True
        return 'Done!'

    def __str__(self):
        if not self.flavor:
            return super().__str__()
        flavor_desc = f'Your best {self.name} is ready! It has: {self.sugar} stickers of sugar, {self.syrup} syrup.'
        if self.cinammon:
            flavor_desc += ', cinammon'
        return flavor_desc


RECIPE = {
    "espresso": {
        'espresso': 30},
    "latte": {
        'espresso': 60,
        'steamed_milk': 120,
        'foamed_milk': 15},
    "macchiato": {
        'espresso': 60,
        'foamed_milk': 15},
    "flat white": {
        'espresso': 60,
        'steamed_milk': 120},
    "dopio": {
        'espresso': 60},
    "cappuccino": {
        'espresso': 60,
        'steamed_milk': 60,
        'foamed_milk': 60},
    "lungo": {
        'espresso': 90},
    "cortado": {
        'espresso': 60,
        'steamed_milk': 60}
}

Зміни по коду:

1. Прибраний сlass FlavorMixin за "відсутністю необхідності" через це частина тестів на CMS не пройде.
2. Замість використання прямого доступу до значення ключа та використання 

3. Замінено конструкції, які використовують умовні оператори для булевих функцій. Заміняються на коротші -> cls.attribute = not cls.attribute.

Приходимо до висновку, що СhatGPT змінює код до меншої кількості команд (див. п.3), а також робить код більш читабельним, замінюючи деякі внутрішні конструкції типу даних на змінні. Проте не додає коментарів та не дотримується в загольному pep8.

Параметри отриманного коду: 1) Час компіляції - 0.001009088с. 2) Кількість використанної пам'яті - 42.5390625 МВ ($\Delta$ = -0.3 МВ)

##### Промпт 2

--> Оптимізуй цей код зі зменшенням кількості пам"яті та часу компіляції.

ChatGPT: --> 
Щоб зменшити кількість пам'яті та часу компіляції, ось деякі можливі оптимізації'яті.

1. Використання слотів: Використання слотів дозволяє зменшити використання пам'яті для об'єктів класу, вказуючи,
   які атрибути можуть бути створені для екземплярів класу. Це корисно, коли є багато екземплярів класу, атрибути яких збігаються.

2. Використання властивостей: Замість прямих атрибутів можна використовувати властивості, що дозволяє здійснювати додаткову логіку при доступі до атрибутів.

3. Використання функцій-генераторів: Функції-генератори дозволяють створювати значення по мірі їх потреби, що допомагає зменшити використання пам'яті.

Отриманий код:

In [12]:
class Track:
    __slots__ = ['data', 'orders', 'revenue', 'milks', 'bean', 'milk']
    __beans = 5000
    __milk = 20000
    safety = True

    MENU = {
        "espresso": 40,
        "latte": 70,
        "flat white": 70,
        "dopio": 50,
        "cappuccino": 60,
        "lungo": 50,
        "cortado": 55,
        "mocca": 60}

    def __init__(self, data):
        self.data = data
        self.orders = []
        self.revenue = 0
        self.milks = 0
        self.bean = 0
        self.milk = Track.__milk

    def place_order(self, order):
        if Track.safety:
            if isinstance(order, Coffee) and order.name in Track.MENU and order.name in Coffee.recipe:
                recipe = Coffee.recipe[order.name]
                required_milk = max(recipe.get('foamed_milk', 0), recipe.get('steamed_milk', 0))
                if required_milk > self.milk:
                    return "Unfortunately, we don't have enough ingredients."
                order.is_paid = True
                self.bean += order.espresso / 5
                self.revenue += order.price
                self.milks += order.milk
                self.orders.append(order)
                self.beans = max(0, Track.__beans - self.bean)
                self.milk = max(0, Track.__milk - self.milks)
                return 'Done!'
            return "Unfortunately, we don't have such kind of coffee in the menu." if order.name not in Track.MENU else "We can't create anything that is not a Coffee instance."
        return "Unfortunately, now it is not safe to make coffee."

    @classmethod
    def set_limit_milk(cls, amount):
        cls.__milk = amount

    @classmethod
    def change_air_state(cls):
        cls.safety = not cls.safety

    def total_revenue(self):
        return self.revenue

    def total_milk(self):
        return self.milks

    def total_beans(self):
        return self.bean

    def milk_spoil(self, amount):
        self.milk -= amount


class Coffee:
    __slots__ = ['name', 'count', 'espresso', 'steamed_milk', 'foamed_milk', 'milk', 'price', 'is_paid']
    recipe = {}

    def __init__(self, name, count=1):
        self.name = name
        self.count = count
        self.is_paid = False
        if self.name in Coffee.recipe and self.name in Track.MENU:
            recipe = Coffee.recipe[self.name]
            self.espresso = recipe['espresso'] * self.count
            self.steamed_milk = recipe.get('steamed_milk', 0) * self.count
            self.foamed_milk = recipe.get('foamed_milk', 0) * self.count
            self.milk = self.steamed_milk + self.foamed_milk
            self.price = Track.MENU[self.name] * self.count

    def __str__(self):
        if not Coffee.recipe:
            return 'Order cannot be created. Recipe has not been set.'
        return f'Order "{self.count} {self.name}" is created.' if self.is_paid else f'Preparing {self.count} {self.name}...'

    def __repr__(self):
        return f'{self.count} {self.name}'

    @classmethod
    def set_recipe(cls, recipe):
        cls.recipe = recipe

class FlavorMixin:
    pass


class CustomCoffee(Coffee):
    __slots__ = ['flavor', 'sugar', 'cinammon', 'syrup']
    
    def __init__(self, name, count=1):
        super().__init__(name, count)
        self.flavor = False

    def add_flavor(self, sugar, cinammon, syrup):
        if not self.is_paid:
            return 'Please, pay for it.'
        self.sugar = sugar * self.count
        self.cinammon = cinammon
        self.syrup = syrup
        self.flavor = True
        return 'Done!'

    def __str__(self):
        if not self.flavor:
            return super().__str__()
        flavor_desc = f'Your best {self.name} is ready! It has: {self.sugar} stickers of sugar, {self.syrup} syrup.'
        if self.cinammon:
            flavor_desc += ', cinammon'
        return flavor_desc


RECIPE = {
    "espresso": {
        'espresso': 30},
    "latte": {
        'espresso': 60,
        'steamed_milk': 120,
        'foamed_milk': 15},
    "macchiato": {
        'espresso': 60,
        'foamed_milk': 15},
    "flat white": {
        'espresso': 60,
        'steamed_milk': 120},
    "dopio": {
        'espresso': 60},
    "cappuccino": {
        'espresso': 60,
        'steamed_milk': 60,
        'foamed_milk': 60},
    "lungo": {
        'espresso': 90},
    "cortado": {
        'espresso': 60,
        'steamed_milk': 60}
}

Coffee.set_recipe(RECIPE)


Параметри отриманного коду: 1. Час компіляції - 0.001014025с 2. Кількість пам"яті, що займає - 42.7109375 МВ ($\Delta$ = -0.1 МВ - не сильно...)

Зміни у коді: перехід до слотів для урізання створення нових комірок пам"яті.

##### Промпт 3
--> Оптимізуй цей код зі зменшенням кількості пам"яті та часом компіляції, дотримуючись pep8


--> СhatGPT: 
Добре, ось кілька покращень для вашого коду з урахуванням PEP8, оптимізації пам'яті та швидкості виконанн:одів.



1. Використання lower() для порівняння рядків: Використання методу lower() для порівняння рядків дозволить уникнути проблем, пов'язаних із регістром.

2. Видалення зайвих перевірок: Деякі перевірки можна вилучити, оскільки вони вже виконуються в інших частинах коду.


3. Короткіші умови: Використання коротших умов може полегшити читання коду.


4. Видалення зайвих змінних: Деякі змінні можна вилучити, оскільки вони використовуються лише для проміжних обчислень.


5. Кешування результатів: Для ефективності можна кешувати результати обчислень, які не змінюються протягом викликів методів.

Отриманий код: 

In [16]:
class Track:
    __beans = 5000
    __milk = 20000
    safety = True

    MENU = {
        "espresso": 40,
        "latte": 70,
        "flat white": 70,
        "dopio": 50,
        "cappuccino": 60,
        "lungo": 50,
        "cortado": 55,
        "mocca": 60}

    def __init__(self):
        self.orders = []
        self.revenue = 0
        self.milk = Track.__milk
        self.beans = Track.__beans

    def place_order(self, order):
        if Track.safety:
            if isinstance(order, Coffee) and order.name in Track.MENU and order.name in Coffee.recipe:
                recipe = Coffee.recipe[order.name]
                if ('foamed_milk' in recipe or 'steamed_milk' in recipe) and recipe.get('foamed_milk', 0) > self.milk:
                    return "Unfortunately, we don't have enough ingredients."
                order.is_paid = True
                self.beans -= order.espresso / 5
                self.revenue += order.price
                self.orders.append(order)
                self.milk = Track.__milk - sum(order.milk for order in self.orders)
                self.beans = Track.__beans - sum(order.espresso / 5 for order in self.orders)
                return 'Done!'
            return "Unfortunately, we don't have such kind of coffee in the menu." if order.name not in Track.MENU else \
                "We can't create anything that is not a Coffee instance."
        return "Unfortunately, now it is not safe to make coffee."

    @classmethod
    def set_limit_milk(cls, amount):
        cls.__milk = amount

    @classmethod
    def change_air_state(cls):
        cls.safety = not cls.safety

    def total_revenue(self):
        return self.revenue

    def total_milk(self):
        return sum(order.milk for order in self.orders)

    def total_beans(self):
        return sum(order.espresso / 5 for order in self.orders)


class Coffee:
    recipe = {}

    def __init__(self, name, count=1):
        self.name = name.lower()
        self.count = count
        self.price = Track.MENU.get(self.name, 0) * self.count
        self.espresso = 0
        self.milk = {'steamed_milk': 0, 'foamed_milk': 0}

        if self.name in Coffee.recipe:
            recipe = Coffee.recipe[self.name]
            self.espresso = recipe.get('espresso', 0) * self.count
            for milk_type in ['steamed_milk', 'foamed_milk']:
                if milk_type in recipe:
                    self.milk[milk_type] = recipe[milk_type] * self.count
        self.is_paid = False

    def __str__(self):
        if Coffee.recipe:
            if self.name in Coffee.recipe:
                return f'Order "{self.count} {self.name}" is created.' if not self.is_paid else f'Preparing {self.count} {self.name}...'
            return "Order cannot be created. We don't have recipe for it."
        return 'Order cannot be created. Recipe has not been set.'

    def __repr__(self):
        return f'{self.count} {self.name}'

    @classmethod
    def set_recipe(cls, recipe):
        cls.recipe = {name.lower(): value for name, value in recipe.items()}

    def __eq__(self, other):
        return self.name == other.name and self.count == other.count

class FlavorMixin:
    pass
    
class CustomCoffee(Coffee, FlavorMixin):
    def __init__(self, name, count=1):
        super().__init__(name, count)
        self.flavor = False
        self.sugar = 0
        self.cinammon = False
        self.syrup = 0

    def add_flavor(self, sugar, cinammon, syrup):
        if not self.is_paid:
            return 'Please, pay for it.'
        self.sugar = sugar * self.count
        self.cinammon = cinammon
        self.syrup = syrup
        self.flavor = True
        return 'Done!'

    def __str__(self):
        if self.flavor:
            return f'Your best {self.name} is ready! It has: {self.sugar} stickers of sugar, {"cinammon, " if self.cinammon else ""}{self.syrup} syrup.'
        return super().__str__()

    def __repr__(self):
        return f'{self.count} custom {self.name}'


RECIPE = {
    "espresso": {
        'espresso': 30},
    "latte": {
        'espresso': 60,
        'steamed_milk': 120,
        'foamed_milk': 15},
    "macchiato": {
        'espresso': 60,
        'foamed_milk': 15},
    "flat white": {
        'espresso': 60,
        'steamed_milk': 120},
    "dopio": {
        'espresso': 60},
    "cappuccino": {
        'espresso': 60,
        'steamed_milk': 60,
        'foamed_milk': 60},
    "lungo": {
        'espresso': 90},
    "cortado": {
        'espresso': 60,
        'steamed_milk': 60}}


Параметри коду: 1. Час компіляції - 0.0010514125с 2. Кількість пам"яті - 42.9375 МВ ( ($\Delta$ = +0.1 МВ - він збільший пам"ять...))

Тобто можна прийти до висновку, що при запиті з проханням зменшити кількість пам"яті, ШІ не перевіряє кількість пам"яті, яка була раніше, та яку він отримав, але використовує ідейні прарадгими, що мали б зменшити пам"ять. 

##### Висновки по Сafeteria:

ШІ робить код читабельнішим в наступному розумінні: він спрощує конструкції, які можуть бути замінені стандартними функціями, а також замінює читабельність конструкцій, вводячи змінні, що описують певну інформацію стосовно змінної. Щодо оптимізації по часу, то по трьом промптам можна сказати, що він зменшив час, проте на 5% швидше, ніж він був раніше. Сам ж ШІ не вміє оптимізовувати задачі по пам"яті, оскільки в перший раз відбулось зменшення пам"яті на 0.1 МВ, а у іншому випадку він її збільшив на 0.1 МВ, тобто, виходячи з цього, можна прийти до висновку, що він не робить перевірку пам"яті, що займає, а лише користується ідеями, що мусили б зменшити пам"ять теоретично. Також ШІ без прохання дотримання assert 

Зауваження: у звіті не використовувалось прохання написати коментарі та тести, проте ШІ з цим чудово справляється. От приклад: 

In [18]:
class Track:
    __beans = 5000  # Приватне поле для зберігання кількості зерен кави
    __milk = 20000  # Приватне поле для зберігання кількості молока
    safety = True   # Поле, що вказує на наявність режиму безпеки

    MENU = {  # Меню напоїв та їх ціни
        "espresso": 40,
        "latte": 70,
        "flat white": 70,
        "dopio": 50,
        "cappuccino": 60,
        "lungo": 50,
        "cortado": 55,
        "mocca": 60}

    def __init__(self):
        # Ініціалізація змінних для відстеження замовлень та доходу
        self.orders = []
        self.revenue = 0
        # Початкові значення молока та зерен кави
        self.milk = Track.__milk
        self.beans = Track.__beans

    def place_order(self, order):
        if Track.safety:
            # Перевірка наявності інгредієнтів та правильності замовлення
            if isinstance(order, Coffee) and order.name in Track.MENU and order.name in Coffee.recipe:
                recipe = Coffee.recipe[order.name]
                if ('foamed_milk' in recipe or 'steamed_milk' in recipe) and recipe.get('foamed_milk', 0) > self.milk:
                    return "Unfortunately, we don't have enough ingredients."
                # Виконання замовлення та оновлення інформації про склад
                order.is_paid = True
                self.beans -= order.espresso / 5
                self.revenue += order.price
                self.orders.append(order)
                self.milk = Track.__milk - sum(order.milk for order in self.orders)
                self.beans = Track.__beans - sum(order.espresso / 5 for order in self.orders)
                return 'Done!'
            # Повідомлення про неможливість виконання замовлення
            return "Unfortunately, we don't have such kind of coffee in the menu." if order.name not in Track.MENU else \
                "We can't create anything that is not a Coffee instance."
        # Повідомлення про небезпечний режим
        return "Unfortunately, now it is not safe to make coffee."

    @classmethod
    def set_limit_milk(cls, amount):
        cls.__milk = amount

    @classmethod
    def change_air_state(cls):
        cls.safety = not cls.safety

    def total_revenue(self):
        return self.revenue

    def total_milk(self):
        return sum(order.milk for order in self.orders)

    def total_beans(self):
        return sum(order.espresso / 5 for order in self.orders)


class Coffee:
    recipe = {}  # Словник з рецептами кави

    def __init__(self, name, count=1):
        # Ініціалізація кави з вказаною назвою та кількістю
        self.name = name.lower()  # Перетворення назви на нижній регістр
        self.count = count
        # Обчислення ціни та кількості інгредієнтів
        self.price = Track.MENU.get(self.name, 0) * self.count
        self.espresso = 0
        self.milk = {'steamed_milk': 0, 'foamed_milk': 0}

        if self.name in Coffee.recipe:
            recipe = Coffee.recipe[self.name]
            self.espresso = recipe.get('espresso', 0) * self.count
            for milk_type in ['steamed_milk', 'foamed_milk']:
                if milk_type in recipe:
                    self.milk[milk_type] = recipe[milk_type] * self.count
        self.is_paid = False

    def __str__(self):
        # Формування рядка представлення замовлення
        if Coffee.recipe:
            if self.name in Coffee.recipe:
                return f'Order "{self.count} {self.name}" is created.' if not self.is_paid else f'Preparing {self.count} {self.name}...'
            return "Order cannot be created. We don't have recipe for it."
        return 'Order cannot be created. Recipe has not been set.'

    def __repr__(self):
        return f'{self.count} {self.name}'

    @classmethod
    def set_recipe(cls, recipe):
        cls.recipe = {name.lower(): value for name, value in recipe.items()}

    def __eq__(self, other):
        # Порівняння замовлень
        return self.name == other.name and self.count == other.count


class CustomCoffee(Coffee):
    def __init__(self, name, count=1):
        # Ініціалізація кастомного замовлення
        super().__init__(name, count)
        self.flavor = False
        self.sugar = 0
        self.cinammon = False
        self.syrup = 0

    def add_flavor(self, sugar, cinammon, syrup):
        # Додавання смаку до замовлення
        if not self.is_paid:
            return 'Please, pay for it.'
        self.sugar = sugar * self.count
        self.cinammon = cinammon
        self.syrup = syrup
        self.flavor = True
        return 'Done!'

    def __str__(self):
        # Формування рядка представлення кастомного замовлення
        if self.flavor:
            return f'Your best {self.name} is ready! It has: {self.sugar} stickers of sugar, {"cinammon, " if self.cinammon else ""}{self.syrup} syrup.'
        return super().__str__()

    def __repr__(self):
        return f'{self.count} custom {self.name}'


RECIPE = {  # Рецепти кави
    "espresso": {
        'espresso': 30},
    "latte": {
        'espresso': 60,
        'steamed_milk': 120,
        'foamed_milk': 15},
    "macchiato": {
        'espresso': 60,
        'foamed_milk': 15},
    "flat white": {
        'espresso': 60,
        'steamed_milk': 120},
    "dopio": {
        'espresso': 60},
    "cappuccino": {
        'espresso': 60,
        'steamed_milk': 60,
        'foamed_milk': 60},
    "lungo": {
        'espresso': 90},
    "cortado": {
        'espresso': 60,
        'steamed_milk': 60}
}

Coffee.set_recipe(RECIPE)  # Встановлення рецептів для кави